In [4]:
# Import the required modules
import json
import csv
import requests
import time
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
from azure.ai.textanalytics import TextAnalyticsApiVersion
from azure.core.exceptions import HttpResponseError

# Define the Azure Text Analytics and Translator credentials
text_analytics_key = "0a4065d87f7a453498faa85ad8c47bf0"
text_analytics_endpoint = "https://azureapi.cognitiveservices.azure.com/"
translator_key = "600a4d74fd7c4962a6111fa03c4d4a7e"
translator_endpoint = "https://api.cognitive.microsofttranslator.com/"

# Create the Text Analytics and Translator clients
text_analytics_client = TextAnalyticsClient(
    endpoint=text_analytics_endpoint,
    credential=AzureKeyCredential(text_analytics_key),
    #api_version=TextAnalyticsApiVersion.V3_0_0 # Use the latest preview version for Text Analytics for health
)
translator_client = requests.Session()
translator_client.headers.update({
    'Ocp-Apim-Subscription-Key': translator_key,
    'Ocp-Apim-Subscription-Region': "eastus2",
    'Content-type': 'application/json'
})

# Define the input and output files
input_file = "input.json"
output_file = "output.csv"

# Define the rate limit for the free tier (5 requests per second)
rate_limit = 5

# Define a helper function to translate a text to English
def translate_to_english(text):
    # Construct the request body
    body = [{
        'text': text
    }]
    # Send the request to the Translator service
    response = translator_client.post(translator_endpoint + '/translate?api-version=3.0&to=en', json=body)
    # Parse the response
    result = response.json()
    # Return the translated text
    return result[0]['translations'][0]['text']

# Open the input file and load the JSON data
with open(input_file, 'r') as f:
    data = json.load(f)

# Open the output file and create a CSV writer
with open(output_file, 'w') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['id', 'text', 'language', 'key_phrases'])

    # Define the list of ids to read
    ids = [76, 157, 127, 201, 478]

    # Loop through the ids
    for id in ids:
        # Get the text from the data using the id as the key
        text = data[str(id)]

        # Detect the language of the text
        try:
            response = text_analytics_client.detect_language(documents=[text])
            language = response[0].primary_language.name
        except HttpResponseError as e:
            print(f'Error detecting language for id {id}: {e}')
            language = 'Unknown'

        # If the language is not English, translate the text to English
        if language != 'English':
            text = translate_to_english(text)

        # Extract the key phrases from the text
        try:
            response = text_analytics_client.extract_key_phrases(documents=[text])
            key_phrases = ', '.join(response[0].key_phrases)
        except HttpResponseError as e:
            print(f'Error extracting key phrases for id {id}: {e}')
            key_phrases = ''

        # Write the output row
        writer.writerow([id, text, language, key_phrases])

        # Wait for 1/rate_limit seconds to avoid exceeding the free tier limit
        time.sleep(1/rate_limit)
